# Task: Build Transformer (TF) for implementing reverse_list

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

## Here we show how we can implement reverse_list() function with looped TF. 

The input/output of the reverse_list function we are implementing is:
- Input: an array of scalars, denoted as input_list = [a_1, ..., a_L]
- Output: the reverse of the input, which is output_list = [a_L, ..., a_1]

This jupyter notebook is organized as below:

1. We first specify 
- the memory & SUBLEQ commands to define the reverse_list function 
- the parameters used for building the looped TF 

2. We run line_search with SUBLEQ commands manually (without using TF)

3. We run line_search with SUBLEQ commands by Looped TF

4. We check whether our Looped TF implementation result matches with ground truth value

## 1. Setup

### Specify the parameters for the program reverse_list

In [2]:
input_list = [-7, -6, -5, -4, 3, 2, 1] # put the list we want to reverse
L = len(input_list)

In [3]:
# write subleq commands & initial memory for the given input_ftrl
import csv

## memory (written in "name=initial_value" format)
# [M_1] MINUS_ONE = -1 (we set a constant for implementing EOF)
# [M_2] ZERO = 0 (we set a constant for implementing EOF)
# [M_3] a_1 = input_list[0]
# ...
# [M_{L+2}] a_L = input_list[L-1]
# [M_{L+3}] temp_1 = 0
# [M_{L+4}] temp_2 = 0
mems = np.concatenate((np.array([-1, 0]), 
                      np.asarray(input_list), 
                      np.array([0, 0]))).reshape(-1, 1) 
mems_path = 'inputs/reverse_list_init_mem.csv'
pd.DataFrame(mems).to_csv(mems_path, header  = ['mem'], index=False)    

## commands 
# NOTE: subleq a,b,c does two things:
# 1. mem[b] = mem[b] - mem[a]
# 2. if mem[b] <= 0: goto instruction c
#   else: goto next instruction

# [C_1] subleq a_1, temp_1, C_2       (temp_1 has -a_1)
# [C_2] subleq a_L, temp_2, C_3       (temp_2 has -a_L) 
# [C_3] subleq a_1, a_1, C_4          (a_1 has zero)
# [C_4] subleq a_L, a_L, C_5          (a_L has zero)
# [C_5] subleq temp_2, a_1, C_6       (a_1 has temp_2)
# [C_6] subleq temp_1, a_L, C_7       (a_L has temp_1)
# [C_7] subleq temp_1, temp_1, C_8    (temp_1 has 0)
# [C_8] subleq temp_2, temp_2, C_9    (temp_2 has 0)
# ... (concatenate this for i=1, ..., floor(L/2), to swap a_i and a_{L-i})
# [C_{8*floor(L/2)}] subleq temp_2, temp_2, C_{8*floor(L/2)+1}
# [C_{8*floor(L/2)+1}] subleq ZERO, MINUS_ONE, C_{8*floor(L/2)+1} (EOF)

cmds = []
for l in range(int(np.floor(L/2))):
    cmds.append([3+l,L+3,8*l+2])   
    cmds.append([L+2-l,L+4,8*l+3]) 
    cmds.append([3+l,3+l,8*l+4]) 
    cmds.append([L+2-l,L+2-l,8*l+5]) 
    cmds.append([L+4,3+l,8*l+6]) 
    cmds.append([L+3,L+2-l,8*l+7])     
    cmds.append([L+3,L+3,8*l+8])
    cmds.append([L+4,L+4,8*l+9])
cmds.append([2,1,8*int(np.floor(L/2))+1])
cmds = np.asarray(cmds)
#cmds.shape

### Specify the parameters for defining the matrix X (input to TF)

In [4]:
num_cmds = cmds.shape[0]
num_mems = mems.shape[0]
s = 2 # s: number of columns in X for scratchpad (larger than 1 is enough?)
m = num_mems # m: number of columns in X for memory
n = s + m + num_cmds # n: number of columns in X in total (for scratchpad, memory and commands)
logn = int(np.ceil(np.log2(n)))

# decide N based on the element with the largest magnitude 
max_element = np.max(np.abs(input_list))
N = int(np.floor(np.log2(max_element)))+3 # N: number of bits used to represent the integer values in each memory element


# Specify the number of rows of input X
# nrows_list: list of number of rows for each block (cmds, memory, scratchpad, program counter, positional encoding, buffer, indicator)
# row_idx_list: list of row index each block starts (memory, scratchpad, program counter, positional encoding, buffer) except cmds & indicator
from utils import get_nrows_subleq, get_row_idx_list_subleq
nrows_cmds, nrows_memory, nrows_scratchpad, nrows_pc, nrows_pos_enc, nrows_buffer = get_nrows_subleq(logn, N) 
num_rows_X = nrows_cmds + nrows_memory + nrows_scratchpad + nrows_pc + nrows_pos_enc + nrows_buffer + 1 
nrows_list = [nrows_cmds, nrows_memory, nrows_scratchpad, nrows_pc, nrows_pos_enc, nrows_buffer, 1] 
row_idx_list = get_row_idx_list_subleq(nrows_list) 
idx_memory, idx_scratchpad, idx_pc, idx_pos_enc, idx_buffer = row_idx_list

### revise & save the cmds.csv 

In [5]:
## add -1 to the address info (mem[a], mem[b], next_cmd) in SUBLEQ cmds
cmds = cmds - 1

## change the index of memory & command in cmds.txt [i -> s+i] [j -> s+m+j]
cmds[:, :2] = cmds[:, :2] + s
cmds[:, 2] = cmds[:, 2] + s + m

In [6]:
cmds_path = 'inputs/reverse_list_cmds.csv'
pd.DataFrame(cmds).to_csv(cmds_path, header  = ['a','b','c'], index=False)    

### Load input text files: (1) the subleq commands, (2) the registers

In [7]:
# load the input files 
cmds_filename = 'inputs/reverse_list_cmds.csv'
cmds_df = pd.read_csv(cmds_filename)
cmds = cmds_df.to_numpy()
mem_filename = 'inputs/reverse_list_init_mem.csv'
mem_df = pd.read_csv(mem_filename)
mem = mem_df.to_numpy().reshape(-1,)

In [8]:
# check the validity of input files
for i in range(len(cmds)):
    #print(i)
    (cmd_a, cmd_b, cmd_c) = cmds[i]
    assert(cmd_a >= s)   # a, b \in [s:s+m] 
    assert(cmd_a < s+m)   
    assert(cmd_b >= s)
    assert(cmd_b < s+m)    
    assert(cmd_c >= s+m) # c \in [s+m:n]
    assert(cmd_c < n)        
assert(len(mem) == m) # mem should have $m$ elements

for i in range(len(mem)):
    assert(mem[i] <= 2**(N-2)-1)
    assert(mem[i] >= -2**(N-2))    
    


### Define the matrix X

In [9]:
from utils import init_input
X, _ = init_input(s,m,n,logn,N,num_rows_X,cmds,nrows_list,row_idx_list,opt=None,mem_given=mem)

## 2. Run factorial (using subleq) manually

In [10]:
num_loops = 8*int(np.floor(L/2))+2 
from utils import run_manual_subleq
manual_subleq_results = run_manual_subleq(cmds, mem, s, m, n, N, num_loops=num_loops)
#manual_subleq_results

In [11]:
header = ['mem[0]']
for i in range(m-2):
    header.append('...')
header = np.append(header, ['mem[m-1]', 'a', 'b', 'c', 'mem[a]', 'mem[b]', 'mem[b]-mem[a]', 'flag', 'p-next'])
manual_subleq_results_df = pd.DataFrame(manual_subleq_results, columns=header)

## 3. Run SUBLEQ using our TF architecture

In [12]:
from subleq import read_inst, read_mem, subtract_mem, write_mem, conditional_branching, error_correction
import os
our_subleq_results = []
our_curr_result = [] 
lam=100 # lambda used for softmax in TF # need to increase as input_fctl increases?

for i in range(num_loops):
    #print('we are in loop ', i)
    
    # Step 1. read instruction & check a, b, c 
    X1, our_curr_result, TF_read_inst = read_inst(X,s,m,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 2. read memory & check mem[a], mem[b]
    X2, our_curr_result, TF_mem = read_mem(X1,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 3. subtract memory & check mem[b]-mem[a] 
    X3, our_curr_result, *TF_subtract_mem = subtract_mem(X2,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 4. write memory 
    X4, our_curr_result, TF_write_mem = write_mem(X3,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 5. conditional branching & check flag, p_{next}
    X5, our_curr_result, *TF_cond_branch = conditional_branching(X4,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 
    # Step 6. error correction        
    X6, our_curr_result, TF_err_corr = error_correction(X5,n,logn,num_rows_X,N,i+1,nrows_list,row_idx_list,None,our_curr_result,lam) 

    X = X6 # go to the next loop
    our_subleq_results.append(tuple(our_curr_result))
    #print(our_curr_result)

In [13]:
our_subleq_results_df = pd.DataFrame(our_subleq_results, columns=header)
#our_subleq_results_df

## 4. Compare our TF SUBLEQ result & manual SUBLEQ result

In [14]:
print('original list: ', input_list)
print('applying reverse_list operation')
input_list.reverse()

true_output = tuple(input_list)
manual_output = manual_subleq_results[-1][2:L+2]
TF_output = our_subleq_results[-1][2:L+2]
print(' true output: {} \n manual output: {} \n TF output: {}'.format(true_output, manual_output, TF_output))

if true_output == manual_output and true_output == TF_output:
    print('The reverse_list in TF works properly!')
else:
    print('Check the code again :p')

original list:  [-7, -6, -5, -4, 3, 2, 1]
applying reverse_list operation
 true output: (1, 2, 3, -4, -5, -6, -7) 
 manual output: (1, 2, 3, -4, -5, -6, -7) 
 TF output: (1, 2, 3, -4, -5, -6, -7)
The reverse_list in TF works properly!
